# Markov Decission Problem (MDP) policy evaluaiton methods (state value V and state-action Q values, policy improvement)

### Legacy repo does not work with gymnasium so some installs and hacks are needed

In [2]:
#!pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

In [1]:
#!pip install gym_walk
#!pip install gymnasium

In [1]:
#import numpy as np
#import gymnasium as gym, gym_walk
#import gymnasium as gym
#from gym_walk.envs import WalkEnv

In [2]:
#from gymnasium.envs.registration import register

In [3]:
#register(
#    # One non-terminal states and two terminal
#    # Technically speaking, this is a bandit MDP
#    id='BanditWalk-v0',
#    entry_point='gym_walk.envs:WalkEnv',
#    # left-most and right-most states are terminal
#    kwargs={'n_states': 1, 'p_stay': 0.0, 'p_backward': 0.0},
#    max_episode_steps=100,
#    reward_threshold=1.0,
#    nondeterministic=True,
#)

In [4]:
#register(
#    # One non-terminal states and two terminal
#    # Technically speaking, this is a bandit MDP
#    id='BanditSlipperyWalk-v0',
#    entry_point='gym_walk.envs:WalkEnv',
#    # left-most and right-most states are terminal
#    kwargs={'n_states': 1, 'p_stay': 0.0, 'p_backward': 0.2},
#    max_episode_steps=100,
#    reward_threshold=1.0,
#    nondeterministic=True,
#)

In [5]:
## (Stochastic) Markov Decission Problem: MDP
## s a (p(a|s), s', r, Teminal?)
#P = {
#    0: {
#        0: [(1.0, 0, 0.0, True)],
#        1: [(1.0, 0, 0.0, True)]
#    },
#    1: {
#        0: [(0.8, 0, 0.0, True), (0.2, 2, 1.0, True)],
#        1: [(0.8, 2, 1.0, True), (0.2, 0, 0.0, True)]
#    },
#    2: {
#        0: [(1.0, 2, 0.0, True)],
#        1: [(1.0, 2, 0.0, True)]
#    }
#}

In [6]:
#env = gym.make('BanditWalk-v0')

# Environment setup

### Upon installing gym_walk lib one can actuall directly instantiated envs used in the book by using proper env parameters

In [1]:
import numpy as np
from gym_walk.envs import WalkEnv

from tqdm import tqdm
import matplotlib.pyplot as plt

### Slippery walk 1D AKA two-arm stochastic bandid

In [2]:
env = WalkEnv(n_states=1, p_stay=0.0, p_backward=0.2)

In [6]:
env.P

{0: {0: [(0.8, 0, 0.0, True), (0.0, 0, 0.0, True), (0.2, 1, 0.0, False)],
  1: [(0.8, 1, 0.0, False), (0.0, 0, 0.0, True), (0.2, 0, 0.0, True)]},
 1: {0: [(0.8, 0, 0.0, True), (0.0, 1, 0.0, False), (0.2, 2, 1.0, True)],
  1: [(0.8, 2, 1.0, True), (0.0, 1, 0.0, False), (0.2, 0, 0.0, True)]},
 2: {0: [(0.8, 1, 0.0, False), (0.0, 2, 0.0, True), (0.2, 2, 0.0, True)],
  1: [(0.8, 2, 0.0, True), (0.0, 2, 0.0, True), (0.2, 1, 0.0, False)]}}

## Policy evaluation - iterative value function estimation from MDP

### Trivial random policy for test

In [132]:
pi = lambda x: np.random.randint(2)

In [199]:
# p - policy, a probability distribution over actions give a state
# P - MDP
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    
    # len(P) - dimension of states space
    states_dim = len(P)
    prev_V = np.zeros(states_dim)

    i = 0
    while True:
        i+=1
        V = np.zeros(states_dim)
        for s in range(states_dim):
            # Iterate over each fan-out of actions out of the given state in MDP P under policy p
            for prob, next_state, reward, done in P[s][pi(s)]:
                #print(reward)
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
                #print(V)
        #if np.max(np.abs(V - prev_V)) < theta:
        #    break
        if i >1000:
            break
        #print(V)
        prev_V = V.copy()
    return V

In [200]:
ret_V = [policy_evaluation(pi, env.P) for _ in range(1)]
V_star = np.mean(ret_V, axis=0)

## Policy improvement

In [201]:
def policy_improvment(V, P, gamma=1.0):
    Q = np.zeros((len(P), len(P[0])))
    for s in range(len(P)):
        for a in range(len(P[s])):
            for prob, next_state, reward, done in P[s][a]:
                Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
    new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return new_pi

In [202]:
p_star = policy_improvment(V_star, env.P, gamma=1.0) 

In [215]:
V_tab = [V_star]
p_tab = [pi]

for _ in range(30):
    V_tab += [policy_evaluation(p_tab[-1], env.P)]
    p_tab += [policy_improvment(V_tab[-1], env.P)]

In [223]:
V_tab[-1]

array([0.64, 0.8 , 0.64])

In [227]:
(p_tab[-1](0), p_tab[-1](1), p_tab[-1](2))

(1, 1, 0)

## Policy iteration

In [231]:
# Action space
tuple(env.P[0].keys())

(0, 1)

In [233]:
# Random policy assigning action (from env.P[0].keys())) to each of states (dim = len(env.P)) 
np.random.choice(tuple(env.P[0].keys()), len(env.P))

array([0, 1, 1])

In [234]:
def policy_iteration(P, gamma=1.0, theta=1e-10):
    # Randomly choose action from the allowed set (here (0, 1)) and assign to every state (here (0, 1, 2))
    random_actions = np.random.choice(tuple(P[0].keys()), len(P))
    pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]
    while True:
        # Store previous policy in a dict
        old_pi = {s:pi(s) for s in range(len(P))}
        V = policy_evaluation(pi, P, gamma, theta)
        pi = policy_improvment(V, P, gamma)
        # Break loop if no change can be made - risky for large stochastic envs?
        if old_pi == {s:pi(s) for s in range(len(P))}:
            break
    return V, pi

In [235]:
V_star, pi_star = policy_iteration(env.P)

In [236]:
V_star

array([0.64, 0.8 , 0.64])

### This agrees with the above naive method 

In [238]:
{s:pi_star(s) for s in range(len(env.P))}

{0: 1, 1: 1, 2: 0}

## Value iteration - by using Q-value one can optimize policy and estimate states values in one call, as opposed to two calls in policy iteration above

In [246]:
def value_iteration(P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
        Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
        for s in range(len(P)):
            for a in range(len(P[s])):
                for prob, next_state, reward, done in P[s][a]:
                    # This is the sum over s', the next states fan-out
                    # with corresponding proper transition probabilities and rewards
                    Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
        if np.max(np.abs(V - np.max(Q, axis=1))) < theta:
            break
        V = np.max(Q, axis=1)
    pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return V, pi

In [247]:
V_i, pi_i = value_iteration(env.P)

In [248]:
V_i

array([0.64, 0.8 , 0.64])

In [249]:
pi_i

<function __main__.value_iteration.<locals>.<lambda>(s)>

In [250]:
{s:pi_i(s) for s in range(len(env.P))}

{0: 1, 1: 1, 2: 0}